In [2]:
import pandas as pd
import calendar

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import helper

### Driver Code

In [8]:
greece_df = helper.make_connection('mobility_stats_google', 'country_region_code', 'GR')
print('------------')
greece_df = helper.clean_df(greece_df)

Connecting to the PostgreSQL database...
Connection successful
Version: ('PostgreSQL 14.0, compiled by Visual C++ build 1914, 64-bit',)
Fetching Data from database...
Data Fetched
------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5296 entries, 0 to 5295
Data columns (total 11 columns):
 #   Column                                              Non-Null Count  Dtype         
---  ------                                              --------------  -----         
 0   country_region_code                                 5296 non-null   object        
 1   sub_region_1                                        4634 non-null   object        
 2   sub_region_2                                        0 non-null      object        
 3   metro_area                                          0 non-null      object        
 4   date                                                5296 non-null   datetime64[ns]
 5   retail_and_recreation_percent_change_from_baseline  5274 non-null   float64    

In [4]:
for col in greece_df.columns[3:-2]:
    print(col)
    print(greece_df['date'][greece_df[col].isnull()])
    print('-----')

retail_and_recreation_percent_change_from_baseline
3387   2020-08-17
3388   2020-08-18
3389   2020-08-19
3390   2020-08-20
3391   2020-08-21
3392   2020-08-22
3393   2020-08-23
3394   2020-08-24
3395   2020-08-25
3396   2020-08-26
3397   2020-08-27
3398   2020-08-28
3400   2020-08-30
3401   2020-08-31
3402   2020-09-01
3403   2020-09-02
3404   2020-09-03
3407   2020-09-06
3408   2020-09-07
3409   2020-09-08
3410   2020-09-09
3411   2020-09-10
Name: date, dtype: datetime64[ns]
-----
grocery_and_pharmacy_percent_change_from_baseline
745    2020-08-23
752    2020-08-30
759    2020-09-06
1274   2020-04-12
1401   2020-08-17
          ...    
3410   2020-09-09
3411   2020-09-10
4824   2020-08-23
4831   2020-08-30
4838   2020-09-06
Name: date, Length: 63, dtype: datetime64[ns]
-----
parks_percent_change_from_baseline
739    2020-08-17
740    2020-08-18
741    2020-08-19
742    2020-08-20
743    2020-08-21
          ...    
4838   2020-09-06
4839   2020-09-07
4840   2020-09-08
4841   2020-09-0

In [4]:
greece_df, stats, idx = helper.aggregate_by_group(greece_df, 'month')

In [5]:
greece_df['day_name'] = greece_df['date'].dt.day_name()
day_by_day_trends = greece_df.groupby(by='day_name').mean().reset_index()

day_by_day_trends.loc[:, 'day_name'] = pd.Categorical(day_by_day_trends['day_name'], categories=list(calendar.day_name), ordered=True)
day_by_day_trends.sort_values('day_name', inplace=True)

day_by_day_trends

day_name  retail_and_recreation_percent_change_from_baseline  \
1     Monday                                          -3.509259    
5    Tuesday                                          -7.570106    
6  Wednesday                                          -5.094920    
4   Thursday                                          -6.486631    
0     Friday                                         -11.520000    
2   Saturday                                         -17.163373    
3     Sunday                                         -17.324967    

   grocery_and_pharmacy_percent_change_from_baseline  \
1                                          21.623670   
5                                          23.670213   
6                                          24.799731   
4                                          28.092742   
0                                          23.209115   
2                                          17.290451   
3                                          53.925776   

   parks_percent_change_from_baseline  \
1                           98.845946   
5                           81.141892   
6                           91.472678   
4                           87.657572   
0                           73.219645   
2                           70.510040   
3                           93.131367   

   transit_stations_percent_change_from_baseline  \
1                                       6.627027   
5                                       5.301351   
6                                       7.049180   
4                                       5.627049   
0                                       5.654003   
2                                      18.881879   
3                                      19.895302   

   workplaces_percent_change_from_baseline  \
1                               -22.627632   
5                               -20.901316   
6                               -21.519947   
4                               -21.232713   
0                               -22.533245   
2                               -13.848684   
3                               -10.652632   

   residential_percent_change_from_baseline     month  
1                                  2.706579  6.684211  
5                                  2.982895  6.715789  
6                                  2.997340  6.702128  
4                                  2.893617  6.734043  
0                                  4.410904  6.638298  
2                                  2.373684  6.610526  
3                                  1.538158  6.642105

In [6]:
daily_smoothed_trends = greece_df.groupby(by='date').mean().reset_index()

In [24]:
first_wave = helper.select_daterange(greece_df, '2020-03-23', '2020-05-04')
second_wave = helper.select_daterange(greece_df, '2020-11-07', '2021-03-22')

In [21]:
COLUMN = 'residential_percent_change_from_baseline'
TITLE = helper.TITLES[-1]
ALPHA = 0.1
helper.exponential_moving_average(daily_smoothed_trends, COLUMN, ALPHA, TITLE)

In [42]:
X = "residential_percent_change_from_baseline"
Y = "parks_percent_change_from_baseline"
helper.corr_heatmap(first_wave, X, Y)